In [2]:
from pyhive import presto
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import time

from datetime import datetime, timedelta

import seaborn as sns

conn = presto.connect(
    host='presto.processing.yoda.run', 
    ## presto.processing.yoda.run
    ## bi-presto.serving.data.production.internal
    port=80,
    username='manoj.ravirajan@rapido.bike'
)

In [3]:
start_date = '2022-06-01'
end_date = '2022-06-30'

# Datasets

### Legacy.orders

In [206]:
## legacy_orders

legacy_orders = f"""
select 
    distinct customer
from legacy.orders
where 
    orderdate between date_format( cast('{start_date}' as date), '%Y-%m-%d') 
    and date_format( cast('{end_date}' as date), '%Y-%m-%d')
    and status = 'dropped' 
    and (spdfraud_flag != 1 OR spdfraud_flag IS NULL)
"""

df_legacy_orders = pd.read_sql(legacy_orders, conn)
df_legacy_orders.head()

,customer
0,5e393884ca251bec90555d41
1,62a83d9501632153ce0f3458
2,622c321b89421f23ea5e680b
3,5e0c8f50a10a71228f89738a
4,6030e5404ef13b0baf332d53


In [207]:
df_legacy_orders.shape

(5814814, 1)

### Ride_behaviour

In [5]:
## lifetimefirstridetime

ride_behaviour = f"""
select 
    distinct customerId as customer1,
    1 as flag1
from hive.datasets.customer_ridebehaviour_servicedetail_immutable as crb
where 
    day between cast('{start_date}' as date) - interval '1' day
    and cast('{end_date}' as date) + interval '1' day
    and lifetimefirstridetime between cast('{start_date}' as date)
    and cast('{end_date}' as date)
    and service_name in ('Auto', 'Link')
"""
df_ride_behaviour = pd.read_sql(ride_behaviour, conn)
df_ride_behaviour.head()

,customer1,flag1
0,615137d6f2602c43332c4c2c,1
1,62b6e4207d7967867cf2f232,1
2,5d818047d0286d106d940f94,1
3,62addef17d796733bbec9711,1
4,5b2e06e78d387c42d138440b,1


In [209]:
df_ride_behaviour.shape

(2188389, 2)

In [210]:
## df_last_ride_behaviour

last_ride_behaviour = f"""
select 
    distinct customerId as customer1,
    1 as flag1
from hive.datasets.customer_ridebehaviour_servicedetail_immutable as crb
where 
    day between cast('{start_date}' as date) - interval '1' day
    and cast('{end_date}' as date) + interval '1' day
    and lifetimelastridetime between cast('{start_date}' as date)
    and cast('{end_date}' as date)
    and service_name in ('Auto', 'Link')
"""
df_last_ride_behaviour = pd.read_sql(last_ride_behaviour, conn)
df_last_ride_behaviour.head()

,customer1,flag1
0,626e522ce98a6446b49b26f9,1
1,5e0b5ee789355c2289a3f0f7,1
2,5fcc9f845441f821744535e0,1
3,5d4fcb84bde10e0d54383b27,1
4,62b58fcc8c7d145ffa0639bb,1


In [211]:
df_last_ride_behaviour.shape

(5672221, 2)

### iallocator_segment

In [6]:
iallocator_segment = f"""
select 
    distinct customer_id as customer2,
    1 as flag2
from datasets.iallocator_customer_segments
where 
    -- taxi_recency_segment = 'RECENT' 
    -- and run_date between date_format( cast('{start_date}' as date) - interval '1' day, '%Y-%m-%d') 
    -- and date_format( cast('{end_date}' as date) + interval '1' day, '%Y-%m-%d')
    -- and 
    run_date = date_format( current_date - interval '3' day, '%Y-%m-%d') 
    and taxi_lifetime_first_ride_date between cast('{start_date}' as date)
    and cast('{end_date}' as date)
"""
df_iallocator_segment = pd.read_sql(iallocator_segment, conn)
df_iallocator_segment.head()

,customer2,flag2
0,5fd3847890cced65b0181d98,1
1,5fd6155669bf68a7c3c8e8e1,1
2,5fdb2e88bb140a4f5daff3b3,1
3,5fdf5ddb4319a75c79b28d16,1
4,5fe18a1c54f1336db195acc3,1


In [7]:
df_iallocator_segment.shape

(1504021, 2)

In [214]:
## taxi_recency_segment = 'RECENT' 

iallocator_segment_recent = f"""
select 
    distinct customer_id as customer2,
    1 as flag2
from datasets.iallocator_customer_segments
where 
    taxi_recency_segment = 'RECENT' 
    -- and run_date between date_format( cast('{start_date}' as date) - interval '1' day, '%Y-%m-%d') 
    -- and date_format( cast('{end_date}' as date) + interval '1' day, '%Y-%m-%d')
    and run_date = date_format( current_date - interval '3' day, '%Y-%m-%d') 
    and taxi_lifetime_first_ride_date between cast('{start_date}' as date)
    and cast('{end_date}' as date)
"""
df_iallocator_segment_recent = pd.read_sql(iallocator_segment_recent, conn)
df_iallocator_segment_recent.head()

,customer2,flag2
0,5df3c273d01edf2b243db9d4,1
1,5e424b3cbf7eab0980a91508,1
2,5e16c01c2f1e6d5f59973a6d,1
3,5e4237dca695d732911dcbe1,1
4,5e43c27d6c87f182ce8fd150,1


In [215]:
df_iallocator_segment_recent.shape

(142438, 2)

# Working

In [8]:
df_view_merge = df_ride_behaviour.merge(df_iallocator_segment, left_on='customer1', right_on='customer2', how='left')
df_view_merge

,customer1,flag1,customer2,flag2
0,615137d6f2602c43332c4c2c,1,NaN,NaN
1,62b6e4207d7967867cf2f232,1,62b6e4207d7967867cf2f232,1.0
2,5d818047d0286d106d940f94,1,5d818047d0286d106d940f94,1.0
3,62addef17d796733bbec9711,1,62addef17d796733bbec9711,1.0
4,5b2e06e78d387c42d138440b,1,NaN,NaN
...,...,...,...,...
2188384,62971e61658bb5705b5e8601,1,62971e61658bb5705b5e8601,1.0
2188385,629db18b737cf75b1984194c,1,629db18b737cf75b1984194c,1.0
2188386,62a8bf83016321c9410f9d36,1,62a8bf83016321c9410f9d36,1.0
2188387,6285b9dab5c8526c2fada9cb,1,NaN,NaN


In [9]:
df1 = df_view_merge

In [10]:
df1.nunique()

customer1    2188389
flag1              1
customer2    1464410
flag2              1
dtype: int64

In [11]:
df1.isna().sum()

customer1         0
flag1             0
customer2    723979
flag2        723979
dtype: int64

In [13]:
df1.flag1.fillna(2, inplace=True)
df1.flag2.fillna(2, inplace=True)

In [15]:
df1.groupby(['flag1','flag2']).agg({'customer1':'count','customer2':'count'}).reset_index()

,flag1,flag2,customer1,customer2
0,1,1.0,1464410,1464410
1,1,2.0,723979,0


In [16]:
1464410-723979

740431

# View one (last ride date)

In [165]:
df_view_merge = df_legacy_orders.merge(df_last_ride_behaviour, left_on='customer', right_on='customer1', how='left')

In [174]:
df_view_1st = df_view_merge.merge(df_iallocator_segment, left_on='customer', right_on='customer2', how='left')

In [184]:
df_view_1st

,customer,customer1,flag1,customer2,flag2
0,5cdeb72825ee3218d4cbcb16,5cdeb72825ee3218d4cbcb16,1.0,NaN,2.0
1,61c4c85b0e8de0f4c65db837,61c4c85b0e8de0f4c65db837,1.0,NaN,2.0
2,61297fe8890209e7995a7346,61297fe8890209e7995a7346,1.0,NaN,2.0
3,60f90469d417d431d77d216e,60f90469d417d431d77d216e,1.0,NaN,2.0
4,619cf5b43f25c316f0e5a716,619cf5b43f25c316f0e5a716,1.0,NaN,2.0
...,...,...,...,...,...
6074291,5e1df77a9a0d23428ae26d5c,5e1df77a9a0d23428ae26d5c,1.0,NaN,2.0
6074292,6272b008a5dd062cfca94a15,6272b008a5dd062cfca94a15,1.0,NaN,2.0
6074293,6065346ea2f9c5ea104f88f6,6065346ea2f9c5ea104f88f6,1.0,NaN,2.0
6074294,62c7e2006d169a9a84477683,62c7e2006d169a9a84477683,1.0,62c7e2006d169a9a84477683,1.0


In [185]:
df_view_1 = df_view_1st

In [177]:
df_view_1.nunique()

customer     6074296
customer1    5939098
flag1              1
customer2    1451047
flag2              1
dtype: int64

In [178]:
df_view_1.isna().sum()

customer           0
customer1     135198
flag1         135198
customer2    4623249
flag2        4623249
dtype: int64

In [179]:
df_view_1.flag1.fillna(2, inplace=True)
df_view_1.flag2.fillna(2, inplace=True)

In [181]:
df_view_1.groupby(['flag1','flag2']).agg({'customer':'count','customer1':'count','customer2':'count'}).reset_index()

,flag1,flag2,customer,customer1,customer2
0,1.0,1.0,1401281,1401281,1401281
1,1.0,2.0,4537817,4537817,0
2,2.0,1.0,49766,0,49766
3,2.0,2.0,85432,0,0


# View two (first ride date)

In [216]:
df2_view_merge = df_legacy_orders.merge(df_ride_behaviour, left_on='customer', right_on='customer1', how='left')

In [217]:
df2_view = df2_view_merge.merge(df_iallocator_segment, left_on='customer', right_on='customer2', how='left')

In [218]:
df2_view_2 = df2_view
df2_view_2

,customer,customer1,flag1,customer2,flag2
0,5e393884ca251bec90555d41,NaN,NaN,NaN,NaN
1,62a83d9501632153ce0f3458,62a83d9501632153ce0f3458,1.0,62a83d9501632153ce0f3458,1.0
2,622c321b89421f23ea5e680b,NaN,NaN,NaN,NaN
3,5e0c8f50a10a71228f89738a,NaN,NaN,NaN,NaN
4,6030e5404ef13b0baf332d53,6030e5404ef13b0baf332d53,1.0,NaN,NaN
...,...,...,...,...,...
5814809,6163218b7fefc5caf58dd427,NaN,NaN,NaN,NaN
5814810,5cb1b85254bc7263ff36dc52,NaN,NaN,NaN,NaN
5814811,5dc2c4fb340548492021bf49,NaN,NaN,NaN,NaN
5814812,6288da338f9b43a6ff602d66,6288da338f9b43a6ff602d66,1.0,NaN,NaN


In [219]:
df2_view_2.nunique()

customer     5814814
customer1    2188382
flag1              1
customer2    1503806
flag2              1
dtype: int64

In [220]:
df2_view_2.isna().sum()

customer           0
customer1    3626432
flag1        3626432
customer2    4311008
flag2        4311008
dtype: int64

In [221]:
df2_view_2.flag1.fillna(2, inplace=True)
df2_view_2.flag2.fillna(2, inplace=True)

In [222]:
df2_view_2.groupby(['flag1','flag2']).agg({'customer':'count','customer1':'count','customer2':'count'}).reset_index()

,flag1,flag2,customer,customer1,customer2
0,1.0,1.0,1464197,1464197,1464197
1,1.0,2.0,724185,724185,0
2,2.0,1.0,39609,0,39609
3,2.0,2.0,3586823,0,0


In [223]:
df2_view_2[(df2_view_2['flag1'].isin([2])) &  (df2_view_2['flag2'].isin([1])) ].head()

,customer,customer1,flag1,customer2,flag2
1949,62a8932255073672d71052a6,NaN,2.0,62a8932255073672d71052a6,1.0
1965,62930be6e7adcd3eff511452,NaN,2.0,62930be6e7adcd3eff511452,1.0
2081,62bd1af17d79672317f73ccb,NaN,2.0,62bd1af17d79672317f73ccb,1.0
2261,62b287397d79670b5befe313,NaN,2.0,62b287397d79670b5befe313,1.0
2275,60cc6702f5cf11046891df8c,NaN,2.0,60cc6702f5cf11046891df8c,1.0


# OLD

In [68]:
code = f"""

with orders as (
select 
    distinct customer
from legacy.orders
where 
    orderdate between date_format( cast('{start_date}' as date), '%Y-%m-%d') 
    and date_format( cast('{end_date}' as date), '%Y-%m-%d')
    and status = 'dropped' 
    and spdfraud_flag!=1
),
     
ridebehaviour as (
select 
    distinct customerId as customer,
    1 as flag
from hive.datasets.customer_ridebehaviour_servicedetail_immutable as crb
where 
    day between cast('{start_date}' as date) - interval '1' day
    and cast('{end_date}' as date) + interval '1' day
    and service_name in ('Auto', 'Link')
),

segment as (
select 
    distinct customer_id as customer,
    1 as flag
from datasets.iallocator_customer_segments
where 
    taxi_recency_segment = 'RECENT' 
    and run_date between date_format( cast('{start_date}' as date) - interval '1' day, '%Y-%m-%d') 
    and date_format( cast('{end_date}' as date) + interval '1' day, '%Y-%m-%d')
) 

select 
    count(distinct orders.customer) legacy,
    count(distinct ridebehaviour.customer) ridebehaviour,
    count(distinct segment.customer) segment,

    case when ridebehaviour.flag = 1 then 1
    else 0 end as ridebehaviour_flag,

    case when segment.flag = 1 then 1
    else 0 end as segment_flag

from orders
left join ridebehaviour on orders.customer = ridebehaviour.customer 
left join segment on ridebehaviour.customer = segment.customer 

group by 4,5
"""

In [69]:
df_code = pd.read_sql(code, conn)
df_code

,legacy,ridebehaviour,segment,ridebehaviour_flag,segment_flag
0,1807,0,0,0,0
1,7081,7081,0,1,0
2,6065408,6065408,6065408,1,1


## Testing

In [29]:
test = f"""
with orders as (select distinct customer from 
legacy.orders
where  status = 'dropped'
--and customer = '57e7742d483bae671710e412'
and orderdate between '2022-06-01' and '2022-06-30'
AND spdfraud_flag!=1),
idebehaviour as (select distinct customerId as customer, 1 as flag
            from hive.datasets.customer_ridebehaviour_servicedetail_immutable as crb
            where 
                day between cast('{start_date}' as date) - interval '1' day
                and cast('{end_date}' as date) + interval '1' day),
            
base as (select orders.customer, case when flag = 1 then 1 else 0 end as flag from orders left join idebehaviour
on orders.customer = idebehaviour.customer)
select count(distinct customer), count(*) from base where flag = 0
"""

In [30]:
df_test = pd.read_sql(test, conn)
df_test

,_col0,_col1
0,3810,3810


In [31]:
test1 = f"""
with orders as (select distinct customer from 
legacy.orders
where  status = 'dropped'
--and customer = '57e7742d483bae671710e412'
and orderdate between '2022-06-01' and '2022-06-30'
AND spdfraud_flag!=1),
segment as (select distinct customer_id as customer,1 as flag  from datasets.iallocator_customer_segments
            where taxi_recency_segment = 'RECENT'
            --and customer_id = '57e7742d483bae671710e412'
            and run_date between '2022-05-31' and '2022-07-01'),
            
base as (select orders.customer, case when flag = 1 then 1 else 0 end as flag from orders left join segment
on orders.customer = segment.customer)
select count(distinct customer), count(*) from base where flag = 0
"""

In [32]:
df_test1 = pd.read_sql(test1, conn)
df_test1

,_col0,_col1
0,40106,40106


In [39]:
test2 = f"""
with orders as (select distinct customer from 
legacy.orders
where  status = 'dropped'
--and customer = '57e7742d483bae671710e412'
and orderdate between '2022-06-01' and '2022-06-30'
AND spdfraud_flag!=1),
segment as (select distinct customer_id as customer,1 as flag  from datasets.iallocator_customer_segments
            where --taxi_recency_segment = 'RECENT'
            --and customer_id = '57e7742d483bae671710e412'
            --and 
            run_date between '2022-05-31' and '2022-07-01'),
            
base as (select orders.customer, case when flag = 1 then 1 else 0 end as flag from orders left join segment
on orders.customer = segment.customer)
select count(distinct customer), count(*) from base where flag = 0
"""

In [40]:
df_test2 = pd.read_sql(test2, conn)
df_test2

,_col0,_col1
0,5705,5705
